In [0]:
from keras.layers import Dense, Conv2D, Input, Lambda, Flatten
from keras.layers import Add, Reshape, MaxPooling2D, Concatenate, RepeatVector, Multiply, dot
from keras.models import Model
from keras import backend as K

import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.engine.topology import Layer
import tensorflow as tf

In [0]:
def attention_util(x, y, axes, normalise=False):
  out = dot(x, y, axes=axes, normalize=normalize)
  out = tf.reduce_sum(out, axis=3)
  return out

In [0]:
def main_attention(l=6*6, d=64*3 , dv=8*3, nv = 8):

    x = Input(shape=(l,d))
    print("input shape - ", x.shape)
    v_in = Dense(d, activation='relu')(x)
    q_in = Dense(d, activation='relu')(x)
    k_in = Dense(d, activation='relu')(x)
    print("Project to linear space:")
    print("v_in - ", v_in.shape)
    print("k_in - ", k_in.shape)
    print("q_in - ", q_in.shape)

    v = Dense(nv*dv, activation = 'relu')(v_in)
    k = Dense(nv*dv, activation = 'relu')(k_in)
    q = Dense(nv*dv, activation = 'relu')(q_in)
    print("v - ", v.shape)
    print("k - ", k.shape)
    print("q - ", q.shape)
   
    v = Reshape([l,nv,dv])(v)  
    k = Reshape([l,nv,dv])(k)  
    q = Reshape([l,nv,dv])(q)
    print("after reshaping")
    print("v - ", v.shape)
    print("k - ", k.shape)
    print("q - ", q.shape)

    # att1 = dot([q,k], axes=(3,3), normalize=True)
    # att1 = tf.reduce_sum(att1, axis=3)
    att_layer = Lambda(lambda x : tf.reduce_sum(dot(x[0], x[1], axes=(3,3), normalize=True), axis=3))
    att1 = att_layer()([q,k])
    print("Dot prod q and k - ", att1.shape)
    att1 = K.softmax(att1, axis=-1)
    print("softmax - ", att1.shape)

    att2 = dot([att1, v], axes=(3,2))
    att2 = tf.reduce_sum(att2, axis=3)
    print("Dot prod v and attention - ", att2.shape)

    out = Reshape([l,d])(att2)
    print("After reshaping - ", out.shape)
    out = Add()([q_in, out])
    print("After adding - ", out.shape)
    #out = Dense(2 * d, activation='relu')(out)
    return Model(inputs=x, outputs=out)
    #return att_model
    # new = Multiply()([q,k])
    # new = Activation('softmax')(new)
   
    # new1 = Multiply()([new,v])
    # new1 = Reshape([l,d])(new1)
    # out = Add()([new1,x])
    # return out

In [0]:
nb_classes = 10
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
X_train = X_train.reshape(60000, 28,28,1)
X_test = X_test.reshape(10000, 28,28,1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train original shape (60000, 28, 28)
y_train original shape (60000,)
Training matrix shape (60000, 28, 28, 1)
Testing matrix shape (10000, 28, 28, 1)


In [0]:
inp = Input((28, 28, 1), name='input_layer')
x = Conv2D(32,(2,2),activation='relu', padding='same')(inp)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64,(2,2),activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
x = Conv2D(64*3,(2,2),activation='relu')(x)
x = Reshape([6*6,64*3])(x)  
att = main_attention(l=6*6, d=64*3 , dv=8*3, nv = 8)
out = att(x)
out = Dense(32, activation = 'relu')(out)
#x = Reshape([6,6,32])(x)  
x = Flatten()(out)
x = Dense(256, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

model = Model(input = inp, output = x)

input shape -  (?, 36, 192)
Project to linear space:
v_in -  (?, 36, 192)
k_in -  (?, 36, 192)
q_in -  (?, 36, 192)
v -  (?, 36, 192)
k -  (?, 36, 192)
q -  (?, 36, 192)
after reshaping
v -  (?, 36, 8, 24)
k -  (?, 36, 8, 24)
q -  (?, 36, 8, 24)


TypeError: ignored